In [6]:
import pymysql
import pandas as pd

In [7]:
# initialize conenction given user and password created in sql file with database tweets
connection = pymysql.connect(host='localhost',
                             user='tweetuser',
                             password='password',
                             db='Tweets')


OperationalError: (1045, "Access denied for user 'tweetuser'@'localhost' (using password: YES)")

In [5]:
# testing out code
cursor = connection.cursor()
sql = "SELECT * FROM Follows;" # automatically assumes count?
cursor.execute(sql)
result = cursor.fetchall() # fetching it gets the real data

# Convert the result to a DataFrame
df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

df.head()

NameError: name 'connection' is not defined

In [2]:
# Establish a database connection
connection = pymysql.connect(
    host="localhost", user="tweetuser", password="password", db="Tweets"
)


def read_tweet_csv(api, csv_data):
    """
    Reads a CSV file and returns a list of Tweet objects

    Args:
        csv_file: A CSV file containing tweet data
    Returns:

        A list of Tweet objects
    """

    for row in csv_data:
        one_tweet = Tweet(
            int(row["tweet_id"]),
            int(row["user_id"]),
            row["tweet_ts"],
            row["tweet_text"],
        )
        api.post_tweet(one_tweet)

    print(row)


def main(csv_file):
    # Open the CSV file once to get the data object,
    # then can get each row with the read_tweet_csv function instead of reading the file each function call
    csv_data = csv.DictReader(open(csv_file))
    # Authenticate
    api = TweetUserAPI(
        os.environ["TWEET_USER"], os.environ["TWEET_PASSWORD"], "tweetuser"
    )

    # Load tweets data into sql database one at a time
    read_tweet_csv(api, csv_data)

# Driver Code
if __name__ == "__main__":
    main(
        csv_file="hw1_data/tweets_sample.csv"
    )  # set filename to tweets to initialize tweets table


NameError: name 'pymysql' is not defined